### Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
from sklearn.metrics import confusion_matrix, classification_report

### Prep Data

In [2]:
# Read in data
df = pd.read_csv("resources/all_data.csv")

# Set index to quarter
df = df.set_index('quarter')

In [3]:
# Save 2019 Q1 & Q2
df_2019 = df.iloc[[-4,-3],:]
df_2019 = df_2019.drop(columns=['recession_actual'])
df_2019

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,4.133333,2.401311,0.083088,0.004646,0.170000,-0.271429,4.5,1.6,104.40334,3783.364,1.458,313000.0,14266.250
2019Q2,255.139333,21340.267,4.7,1255.666667,0.828815,3.500000,2.397813,-0.001457,0.024002,0.213333,0.254902,2.4,1.6,103.20060,3749.471,1.457,322500.0,14511.176


In [4]:
# Drop rows with missing values
df = df.dropna()
df.tail()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,recession_actual,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,
2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,0.0,4.333333,1.448966,0.204683,0.083902,0.596667,-0.113861,6.9,1.8,104.59493,3542.412,1.451,331800.0,13728.357
2018Q2,250.578667,20510.177,7.1,1259.666667,0.589182,0.0,3.833333,1.727176,0.192007,0.075492,0.446667,-0.251397,2.7,2.0,103.33928,3561.592,1.461,315600.0,13939.828
2018Q3,251.828667,20749.752,4.8,1233.000000,0.821959,0.0,3.866667,1.923492,0.113663,0.047184,0.253333,-0.432836,3.3,2.0,103.69309,3683.981,1.462,330900.0,14114.559
2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,0.066218,0.233333,-0.078947,2.8,1.9,105.15026,3725.234,1.462,322800.0,14211.920
2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,0.004646,0.170000,-0.271429,4.5,1.6,104.40334,3783.364,1.458,313000.0,14266.250


### Shift Data

In [5]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [6]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [7]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()

In [8]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [9]:
# Define X
X1 = df_q1.drop(columns=['recession_1q_out'])
X2 = df_q2.drop(columns=['recession_2q_out'])
X3 = df_q4.drop(columns=['recession_4q_out'])

### Split and scale data

In [15]:
# Split data into training and testing
X1_train, X1_test, y1_train, y1_test=train_test_split(X1, y1, train_size=0.8, shuffle=False)
X2_train, X2_test, y2_train, y2_test=train_test_split(X2, y2, train_size=0.8, shuffle=False)
X3_train, X3_test, y3_train, y3_test=train_test_split(X3, y3, train_size=0.8, shuffle=False)

In [16]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

In [17]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

# Scale full X data (no splits)
X1_full_scaled = X1_scaler.transform(X1)
X2_full_scaled = X2_scaler.transform(X2)
X3_full_scaled = X3_scaler.transform(X3)

### Reshape data to fit LSTM format

In [18]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [19]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [20]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

In [21]:
# Reshape X_full
reshaped_X1_full = reshape_data(X1_full_scaled)
reshaped_X2_full = reshape_data(X2_full_scaled)
reshaped_X3_full = reshape_data(X3_full_scaled)

# Build Model

In [22]:
# Initialize model
model = Sequential()

In [23]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

## Train and predict on X1-Y1 data (recession 1 quarter out)

In [25]:
# Fit the model to the training data
model.fit(reshaped_X1_train_scaled, y1_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 28 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
108/108 - 11s - loss: 1.6224 - acc: 0.4074 - val_loss: 0.6767 - val_acc: 0.7500
Epoch 2/100
108/108 - 1s - loss: 1.2371 - acc: 0.4907 - val_loss: 0.6628 - val_acc: 0.7500
Epoch 3/100
108/108 - 1s - loss: 1.2837 - acc: 0.4815 - val_loss: 0.6469 - val_acc: 0.7500
Epoch 4/100
108/108 - 1s - loss: 1.1628 - acc: 0.5093 - val_loss: 0.6319 - val_acc: 0.7500
Epoch 5/100
108/108 - 1s - loss: 0.7947 - acc: 0.6019 - val_loss: 0.6166 - val_acc: 0.7500
Epoch 6/100
108/108 - 1s - loss: 0.7876 - acc: 0.5833 - val_loss: 0.6017 - val_acc: 0.7500
Epoch 7/100
108/108 - 1s - loss: 0.6749 - acc: 0.6204 - val_loss: 0.5899 - val_acc: 0.7500
Epoch 8/100
108/108 - 1s - loss: 0.6641 - acc: 0.7130 - val_loss: 0.5815 - val_acc: 0.7500
Epoch 9/100
108/108 - 1s - loss: 0.8129 - acc: 0.7037 - val_loss: 0.5730 - val_acc: 0.7500
Epoch 10/100
108/108 - 1s - loss: 0.6910 

108/108 - 1s - loss: 0.1756 - acc: 0.9167 - val_loss: 1.2150 - val_acc: 0.7500
Epoch 87/100
108/108 - 1s - loss: 0.1615 - acc: 0.9352 - val_loss: 1.2505 - val_acc: 0.7143
Epoch 88/100
108/108 - 1s - loss: 0.1920 - acc: 0.8981 - val_loss: 1.2764 - val_acc: 0.7500
Epoch 89/100
108/108 - 1s - loss: 0.2085 - acc: 0.8796 - val_loss: 1.2741 - val_acc: 0.7500
Epoch 90/100
108/108 - 1s - loss: 0.2214 - acc: 0.9167 - val_loss: 1.2641 - val_acc: 0.7500
Epoch 91/100
108/108 - 1s - loss: 0.1755 - acc: 0.9259 - val_loss: 1.2713 - val_acc: 0.7500
Epoch 92/100
108/108 - 1s - loss: 0.1681 - acc: 0.9167 - val_loss: 1.2744 - val_acc: 0.7500
Epoch 93/100
108/108 - 1s - loss: 0.1915 - acc: 0.9167 - val_loss: 1.3264 - val_acc: 0.7500
Epoch 94/100
108/108 - 1s - loss: 0.1169 - acc: 0.9352 - val_loss: 1.4070 - val_acc: 0.7500
Epoch 95/100
108/108 - 1s - loss: 0.1723 - acc: 0.9167 - val_loss: 1.4872 - val_acc: 0.7143
Epoch 96/100
108/108 - 1s - loss: 0.1908 - acc: 0.8981 - val_loss: 1.4951 - val_acc: 0.7143
E

In [26]:
# Evaluate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

34/34 - 0s - loss: 0.0184 - acc: 1.0000


In [27]:
# Make predictions using test data
predictions1_class = model.predict_classes(reshaped_X1_test_scaled) # Predicted class

In [30]:
# Compare results
one_qtr_out = pd.DataFrame({"Predicted":predictions1_class, "Actual":y1_test})
one_qtr_out.loc[one_qtr_out["Actual"]==1]

,Predicted,Actual
quarter,,


#### Confusion Matrix on X1-Y1 data (recession 1 quarter out)

In [31]:
# Create confusion matrix on X1 model
con_mat = confusion_matrix(y1_test, predictions1_class)
print(con_mat)

[[34]]


In [32]:
# Score model
print(classification_report(y1_test, predictions1_class))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        34

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



In [26]:
# Save model
# name1 = f"unshuffled-1q-out-{int(time.time())}"
# model.save(f"models/{name1}.h5")

### Predict on 2019

In [33]:
# Scale 2019 data 
scaled_X1_2019 = X1_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X1_2019 = reshape_data(scaled_X1_2019)

# Predict on 2019
pred_X1_2019 = model.predict_proba(reshaped_X1_2019)
print(f"2019Q1 No Recession Probability: {pred_X1_2019[0][0]}")
print(f"2019Q1 Recession Probability: {pred_X1_2019[0][1]}")
print(f"2019Q2 No Recession Probability: {pred_X1_2019[1][0]}")
print(f"2019Q2 Recession Probability: {pred_X1_2019[1][1]}")

2019Q1 No Recession Probability: 0.9998992681503296
2019Q1 Recession Probability: 0.00010077308979816735
2019Q2 No Recession Probability: 0.999792754650116
2019Q2 Recession Probability: 0.0002072450442938134


#### Predict on full X1

In [34]:
pred_X1_full = model.predict_classes(reshaped_X1_full)

# Preview results
X1_full_results = pd.DataFrame({"Predicted":pred_X1_full, "Actual":y1})
X1_full_results.loc[X1_full_results["Actual"]==1]

# Export results for graphing
# X1_full_results.to_csv(f"resources/predictions/X1_NS_{int(time.time())}.csv")

,Predicted,Actual
quarter,,
1979Q1,1,1.0
1979Q2,0,1.0
1979Q3,1,1.0
1979Q4,1,1.0
1980Q1,1,1.0
1981Q1,0,1.0
1981Q2,1,1.0
1981Q3,1,1.0
1981Q4,0,1.0


### Train and predict on X2-Y2 data (recession 2 quarters out)

In [35]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 108 samples, validate on 27 samples
Epoch 1/100
108/108 - 1s - loss: 0.3597 - acc: 0.8704 - val_loss: 1.7089 - val_acc: 0.7407
Epoch 2/100
108/108 - 1s - loss: 0.4484 - acc: 0.9352 - val_loss: 1.7873 - val_acc: 0.7407
Epoch 3/100
108/108 - 1s - loss: 0.2870 - acc: 0.9167 - val_loss: 1.7993 - val_acc: 0.7407
Epoch 4/100
108/108 - 1s - loss: 0.2079 - acc: 0.9352 - val_loss: 1.7941 - val_acc: 0.7407
Epoch 5/100
108/108 - 1s - loss: 0.3562 - acc: 0.9074 - val_loss: 1.7332 - val_acc: 0.7407
Epoch 6/100
108/108 - 1s - loss: 0.2173 - acc: 0.9444 - val_loss: 1.6356 - val_acc: 0.7407
Epoch 7/100
108/108 - 1s - loss: 0.1969 - acc: 0.9352 - val_loss: 1.5076 - val_acc: 0.7407
Epoch 8/100
108/108 - 1s - loss: 0.2095 - acc: 0.9074 - val_loss: 1.3721 - val_acc: 0.7778
Epoch 9/100
108/108 - 1s - loss: 0.1812 - acc: 0.9074 - val_loss: 1.3110 - val_acc: 0.7778
Epoch 10/100
108/108 - 1s - loss: 0.2332 - acc: 0.9167 - val_loss: 1.2923 - val_acc: 0.7778
Epoch 11/100
108/108 - 1s - loss: 0.1947 - a

Epoch 90/100
108/108 - 1s - loss: 0.0430 - acc: 0.9907 - val_loss: 1.4474 - val_acc: 0.7407
Epoch 91/100
108/108 - 1s - loss: 0.0495 - acc: 0.9815 - val_loss: 1.5926 - val_acc: 0.7407
Epoch 92/100
108/108 - 1s - loss: 0.0316 - acc: 1.0000 - val_loss: 1.6906 - val_acc: 0.7407
Epoch 93/100
108/108 - 1s - loss: 0.1237 - acc: 0.9722 - val_loss: 1.7614 - val_acc: 0.7407
Epoch 94/100
108/108 - 1s - loss: 0.0554 - acc: 0.9630 - val_loss: 1.7559 - val_acc: 0.7407
Epoch 95/100
108/108 - 1s - loss: 0.0532 - acc: 0.9907 - val_loss: 1.6445 - val_acc: 0.7407
Epoch 96/100
108/108 - 1s - loss: 0.0324 - acc: 0.9907 - val_loss: 1.4735 - val_acc: 0.7407
Epoch 97/100
108/108 - 1s - loss: 0.0508 - acc: 0.9722 - val_loss: 1.3998 - val_acc: 0.7407
Epoch 98/100
108/108 - 1s - loss: 0.0252 - acc: 0.9907 - val_loss: 1.3801 - val_acc: 0.7407
Epoch 99/100
108/108 - 1s - loss: 0.0256 - acc: 0.9907 - val_loss: 1.3349 - val_acc: 0.7407
Epoch 100/100
108/108 - 1s - loss: 0.0196 - acc: 0.9907 - val_loss: 1.3068 - val

In [36]:
# Evaluate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

34/34 - 0s - loss: 0.0604 - acc: 1.0000


In [37]:
# Make predictions using test data
predictions2_class = model.predict_classes(reshaped_X2_test_scaled)

In [38]:
# Compare results
two_qtrs_out = pd.DataFrame({"Predicted":predictions2_class, "Actual":y2_test})
two_qtrs_out.loc[two_qtrs_out["Actual"]==1]

,Predicted,Actual
quarter,,


#### Confusion Matrix on X2-Y2 data (recession 2 quarters out)

In [39]:
# Create confusion matrix on X2 model
con_mat = confusion_matrix(y2_test, predictions2_class)
print(con_mat)

[[34]]


In [40]:
# Score model
print(classification_report(y2_test, predictions2_class))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        34

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



In [41]:
# Save model
# name2 = f"unshuffled-2q-out-{int(time.time())}"
# model.save(f"models/{name2}.h5")

### Predict on 2019

In [42]:
# Scale 2019 data 
scaled_X2_2019 = X2_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X2_2019 = reshape_data(scaled_X2_2019)

# Predict on 2019
pred_X2_2019 = model.predict_proba(reshaped_X2_2019)
print(f"2019Q1 No Recession Probability: {pred_X2_2019[0][0]}")
print(f"2019Q1 Recession Probability: {pred_X2_2019[0][1]}")
print(f"2019Q2 No Recession Probability: {pred_X2_2019[1][0]}")
print(f"2019Q2 Recession Probability: {pred_X2_2019[1][1]}")

2019Q1 No Recession Probability: 0.839864194393158
2019Q1 Recession Probability: 0.16013580560684204
2019Q2 No Recession Probability: 0.8839673399925232
2019Q2 Recession Probability: 0.11603259295225143


#### Predict on full X2

In [43]:
pred_X2_full = model.predict_classes(reshaped_X2_full)
X2_full_results = pd.DataFrame({"Predicted":pred_X2_full, "Actual":y2})
X2_full_results.loc[X2_full_results["Actual"]==1]
# X2_full_results.to_csv(f"resources/predictions/X2_NS_{int(time.time())}.csv")

,Predicted,Actual
quarter,,
1978Q4,1,1.0
1979Q1,0,1.0
1979Q2,0,1.0
1979Q3,0,1.0
1979Q4,0,1.0
1980Q4,1,1.0
1981Q1,1,1.0
1981Q2,0,1.0
1981Q3,1,1.0


### Train and predict on X3-Y3 data (recession 4 quarters out)

In [44]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, validation_split=0.2, epochs=100, shuffle=False, verbose=2)

Train on 106 samples, validate on 27 samples
Epoch 1/100
106/106 - 1s - loss: 1.1052 - acc: 0.8491 - val_loss: 1.1160 - val_acc: 0.7037
Epoch 2/100
106/106 - 2s - loss: 0.5915 - acc: 0.8585 - val_loss: 1.1128 - val_acc: 0.4444
Epoch 3/100
106/106 - 1s - loss: 0.4928 - acc: 0.8774 - val_loss: 1.8382 - val_acc: 0.3333
Epoch 4/100
106/106 - 1s - loss: 0.3204 - acc: 0.8962 - val_loss: 2.0217 - val_acc: 0.3704
Epoch 5/100
106/106 - 1s - loss: 0.2860 - acc: 0.9151 - val_loss: 1.6799 - val_acc: 0.3704
Epoch 6/100
106/106 - 1s - loss: 0.3483 - acc: 0.8868 - val_loss: 1.2995 - val_acc: 0.2593
Epoch 7/100
106/106 - 1s - loss: 0.2190 - acc: 0.8962 - val_loss: 1.1261 - val_acc: 0.4074
Epoch 8/100
106/106 - 1s - loss: 0.1824 - acc: 0.9340 - val_loss: 1.1570 - val_acc: 0.4074
Epoch 9/100
106/106 - 1s - loss: 0.2422 - acc: 0.9151 - val_loss: 1.2185 - val_acc: 0.3704
Epoch 10/100
106/106 - 1s - loss: 0.1817 - acc: 0.9245 - val_loss: 1.1859 - val_acc: 0.4444
Epoch 11/100
106/106 - 1s - loss: 0.1720 - a

Epoch 90/100
106/106 - 1s - loss: 0.0417 - acc: 0.9906 - val_loss: 2.4135 - val_acc: 0.5556
Epoch 91/100
106/106 - 1s - loss: 0.0236 - acc: 1.0000 - val_loss: 2.5688 - val_acc: 0.4815
Epoch 92/100
106/106 - 1s - loss: 0.0258 - acc: 0.9906 - val_loss: 2.8187 - val_acc: 0.4815
Epoch 93/100
106/106 - 1s - loss: 0.0408 - acc: 0.9811 - val_loss: 3.3271 - val_acc: 0.4074
Epoch 94/100
106/106 - 1s - loss: 0.0162 - acc: 0.9906 - val_loss: 3.5044 - val_acc: 0.3704
Epoch 95/100
106/106 - 1s - loss: 0.0208 - acc: 1.0000 - val_loss: 3.5012 - val_acc: 0.3704
Epoch 96/100
106/106 - 1s - loss: 0.0627 - acc: 0.9528 - val_loss: 3.3328 - val_acc: 0.4074
Epoch 97/100
106/106 - 1s - loss: 0.0531 - acc: 0.9623 - val_loss: 3.2742 - val_acc: 0.4074
Epoch 98/100
106/106 - 1s - loss: 0.0812 - acc: 0.9717 - val_loss: 3.3149 - val_acc: 0.4074
Epoch 99/100
106/106 - 1s - loss: 0.0274 - acc: 0.9906 - val_loss: 3.4435 - val_acc: 0.3704
Epoch 100/100
106/106 - 1s - loss: 0.0311 - acc: 0.9906 - val_loss: 3.4091 - val

In [45]:
# Validate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

34/34 - 0s - loss: 3.5364 - acc: 0.3235


In [46]:
# Make predictions using test data
predictions3_class = model.predict_classes(reshaped_X3_test_scaled)

In [47]:
# Compare results
four_qtrs_out = pd.DataFrame({"Predicted":predictions3_class, "Actual":y3_test})
four_qtrs_out.loc[four_qtrs_out["Actual"]==1]

,Predicted,Actual
quarter,,


#### Confusion Matrix on X3-Y3 data (recession 4 quarters out)

In [48]:
# Create confusion matrix on X3 model
con_mat = confusion_matrix(y3_test, predictions3_class)
print(con_mat)

[[11 23]
 [ 0  0]]


In [49]:
# Score model
print(classification_report(y3_test, predictions3_class))

              precision    recall  f1-score   support

         0.0       1.00      0.32      0.49        34
         1.0       0.00      0.00      0.00         0

    accuracy                           0.32        34
   macro avg       0.50      0.16      0.24        34
weighted avg       1.00      0.32      0.49        34



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [50]:
# Save model
# name3 = f"unshuffled-4q-out-{int(time.time())}"
# model.save(f"models/{name3}.h5")

### Predict on 2019

In [51]:
# Scale 2019 data 
scaled_X3_2019 = X3_scaler.transform(df_2019)

# Reshape 2019 data
reshaped_X3_2019 = reshape_data(scaled_X3_2019)

# Predict on 2019
pred_X3_2019 = model.predict_proba(reshaped_X3_2019)
print(f"2019Q1 No Recession Probability: {pred_X3_2019[0][0]}")
print(f"2019Q1 Recession Probability: {pred_X3_2019[0][1]}")
print(f"2019Q2 No Recession Probability: {pred_X3_2019[1][0]}")
print(f"2019Q2 Recession Probability: {pred_X3_2019[1][1]}")

2019Q1 No Recession Probability: 0.0037298204842954874
2019Q1 Recession Probability: 0.9962701797485352
2019Q2 No Recession Probability: 0.03209241107106209
2019Q2 Recession Probability: 0.9679075479507446


#### Predict on full X3

In [52]:
pred_X3_full = model.predict_classes(reshaped_X3_full)

# Preview results
X3_full_results = pd.DataFrame({"Predicted":pred_X3_full, "Actual":y3})
X3_full_results.loc[X3_full_results["Actual"]==1]

# Export results for graphing
# X3_full_results.to_csv(f"resources/predictions/X3_NS_{int(time.time())}.csv")

,Predicted,Actual
quarter,,
1978Q2,1,1.0
1978Q3,0,1.0
1978Q4,1,1.0
1979Q1,0,1.0
1979Q2,0,1.0
1980Q2,1,1.0
1980Q3,0,1.0
1980Q4,1,1.0
1981Q1,1,1.0
